In [18]:
import os

import arviz as az
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import gaussian_kde

import jax.numpy as jnp
from jax import random, vmap

import numpyro
import numpyro.distributions as dist

if "SVG" in os.environ:
    %config InlineBackend.figure_formats = ["svg"]
az.style.use("arviz-darkgrid")


In [19]:
# Code 3.15
# samples[jnp.argmax(gaussian_kde(samples, bw_method=0.01)(samples))]

# Exercises

In [20]:
p_grid = jnp.linspace(start=0, stop=1, num=1000)
prior = jnp.repeat(1, 1000)
likelihood = jnp.exp(dist.Binomial(total_count=9, probs=p_grid).log_prob(6))
posterior = likelihood * prior
posterior = posterior / jnp.sum(posterior)
samples = p_grid[dist.Categorical(posterior).sample(random.PRNGKey(100), (10000,))]

In [21]:
# 3E1
(samples < .2).sum() / 1e4

DeviceArray(0.001, dtype=float32)

In [22]:
# 3E2
(samples < 0.8).sum() / 1e4

DeviceArray(0.879, dtype=float32)

In [23]:
# 3E3
((0.2 < samples) & (samples < 0.8)) / 1e4

DeviceArray([1.e-04, 1.e-04, 1.e-04, ..., 1.e-04, 1.e-04, 1.e-04], dtype=float32)

In [24]:
# 3E4 - 20% of the posterior probability lies below what value of p?
def infer_bruteforce_lt(samples: type(samples), x: float, rtol: float = 1e-5) -> float: 
    """x percent of posterior probability lies below what value of p?"""
    answers = list()
    for k in jnp.linspace(start=0, stop=1, num=1000): 
        if jnp.isclose((samples < k).sum() / samples.shape[0], x, rtol=rtol): 
            answers.append(k)
    if not answers: 
        raise ValueError("needs to be called with a more permissive tolerance for this data")
    return jnp.array(answers).mean()

infer_bruteforce_lt(samples, 0.2, 1e-2)

DeviceArray(0.5195195, dtype=float32)

In [25]:
# 3E5 - 20% of the posterior probability lies above what value of p?
def infer_bruteforce_gt(samples: type(samples), x: float, rtol: float = 1e-5) -> float: 
    """x percent of posterior probability lies above what value of p?"""
    answers = list()
    for k in jnp.linspace(start=0, stop=1, num=1000): 
        if jnp.isclose((samples > k).sum() / samples.shape[0], x, rtol=rtol): 
            answers.append(k)
    if not answers: 
        raise ValueError("needs to be called with a more permissive tolerance for this data")
    return jnp.array(answers).mean()

infer_bruteforce_gt(samples, 0.2, 1e-2)

DeviceArray(0.7602603, dtype=float32)

In [26]:
# 3E6
numpyro.diagnostics.hpdi(samples, prob=0.66)

array([0.5155155, 0.7847848], dtype=float32)

In [35]:
# 3E7
x, y = jnp.percentile(samples, q=(0.17,0.83))
x, y

(0.2102068, 0.25324494)

In [28]:
# 3M1
p_grid = jnp.linspace(start=0, stop=1, num=1000)
prior = jnp.repeat(1, 1000)
likelihood = jnp.exp(dist.Binomial(total_count=15, probs=p_grid).log_prob(8))
posterior = likelihood * prior
posterior = posterior / jnp.sum(posterior)
samples = p_grid[dist.Categorical(posterior).sample(random.PRNGKey(100), (10000,))]

In [29]:
# 3M2
numpyro.diagnostics.hpdi(samples, prob=0.9)

array([0.33333334, 0.7187187 ], dtype=float32)

In [30]:
# 3M3 Construct a posterior predictive check for this model and data. This means simulate the distribution 
# of samples, averaging over the posterior uncertainty in p. What is the probability of 
# observing 8 water in 15 tosses?

dummy_3M3 <- rbinom(1e5, size=15, prob=p_grid)

NameError: name 'dummy_3M3' is not defined

In [53]:
# 3M6 - Suppose you want to estimate the Earth’s proportion of water very precisely. Specifically, you
# want the 99% percentile interval of the posterior distribution of p to be only 0.05 wide. This means
# the distance between the upper and lower bound of the interval should be 0.05. How many times will
# you have to toss the globe to do this?

def threeM6(percentile_interval: float = 0.99, desired_width: float = 0.05) -> int: 
    p_grid = jnp.linspace(start=0, stop=1, num=1000)
    prior = jnp.repeat(1, 1000)
    
    tosses = 3
    while True: 
        binom = dist.Binomial(total_count=tosses, probs=p_grid)
        likelihood = jnp.exp(dist.Binomial(total_count=tosses, probs=p_grid).log_prob(6))
        posterior = likelihood * prior
        posterior = posterior / jnp.sum(posterior)
        samples = p_grid[dist.Categorical(posterior).sample(random.PRNGKey(100), (10000,))]
        lower, upper = jnp.percentile(samples, q=(0.005, 0.995))
        print(lower, upper)
        if upper - lower <= desired_width: 
            return tosses
        toss += 1

threeM6()

AttributeError: '_DeviceArray' object has no attribute 'log_prob'

In [58]:
dist.Binomial(total_count=7, probs=p_grid).log_prob(6)

DeviceArray([        -inf, -39.49562   , -35.337738  , -32.90595   ,
             -31.180864  , -29.843008  , -28.750084  , -27.826189  ,
             -27.02601   , -26.320322  , -25.689169  , -25.118319  ,
             -24.597263  , -24.118021  , -23.674387  , -23.261448  ,
             -22.87523   , -22.512503  , -22.170572  , -21.847185  ,
             -21.54045   , -21.248728  , -20.970633  , -20.704945  ,
             -20.450613  , -20.206705  , -19.972408  , -19.746996  ,
             -19.529818  , -19.320303  , -19.117924  , -18.922216  ,
             -18.73276   , -18.549162  , -18.371082  , -18.198193  ,
             -18.030205  , -17.86685   , -17.707882  , -17.55307   ,
             -17.402206  , -17.25509   , -17.111551  , -16.971415  ,
             -16.834526  , -16.700735  , -16.56991   , -16.441923  ,
             -16.31665   , -16.193989  , -16.073826  , -15.956065  ,
             -15.840611  , -15.72738   , -15.616283  , -15.507248  ,
             -15.400196  , -15.295